# CC3001 Otoño 2020 Tarea 4
# Generación de código

Profesores: Sección 1 Patricio Poblete / Sección 2 Nelson Baloian

Plazo: Lunes 8 de junio de 2020 a las 23:59

Estudiante: Pía Contreras Guerrero

El objetivo de esta tarea es aprender sobre la manipulación de árboles que representan fórmulas. Hay muchas cosas que uno puede hacer con estos árboles, y en esta tarea nos enfocaremos en particular en algo que los compiladores hacen: la **generación de código** para evaluar fórmulas.

Específicamente, el problema es: dada una fórmula, generar una secuencia de instrucciones elementales que la evalúen. Por ejemplo, dada una fórmula como

```
(a+b)*(c-d)
```

se podría generar código como el siguiente:

```
t1=a+b
t2=c-d
t3=t1*t2
```

Las instrucciones elementales son siempre de la forma ``ti=x op y``, donde ``ti`` es una variable temporal, ``x`` e ``y`` pueden ser variables o números, y ``op`` es un operador. Uno va creando variables ``t1``, ``t2``, ... a medida que las necesita, y la última de ellas es la que almacena el valor final de la fórmula.

Las fórmulas pueden ser arbitrariamente complicadas, como por ejemplo la fórmula

$$
(2-p\,q)\left( \frac{1}{n}+\frac{1}{p+q}\right)
$$

la cual escribimos como

```
(2-p*q)*(1/n+1/(p+q))
```

y cuyo código generado podría ser,

```
t1=p*q
t2=2-t1
t3=1/n
t4=p+q
t5=1/t4
t6=t3+t5
t7=t2*t6
```

El objetivo de esta tarea es que usted implemente una función que, dado un árbol ya construído, lo recorra para generar una secuencia de instrucciones elementales que lo evalúe.

A continuación usted encontrará clases ``Nodoi``, ``Nodoe`` y ``Arbol`` que implementan árboles que representan fórmulas. La clase ``Arbol`` viene con un constructor que recibe un string como parámetro, el cual contiene una fórmula, y este constructor se encarga de transformar la fórmula de string a árbol, de modo que usted no necesita preocuparse de eso. Para simplificar, la fórmula solo podrá contener variables de una sola letra, números de un solo dígito, y sin espacios. Los operadores permitidos son solo suma, resta, multiplicación y división. No hay operador menos unario.

Usted tienen que implementar para la clase ``Arbol`` una función ``codigo`` que al ser invocado genere una secuencia de instrucciones elementales para evaluar la fórmula representada por ese árbol, y la retorne en forma de una lista (de Python).

In [0]:
class Nodoi:
    def __init__(self, izq, info, der):
        self.izq=izq
        self.info=info
        self.der=der
    def postorden(self):
      v1=self.izq.postorden()   
      v2=self.der.postorden()
      global lista
      global contador
      variable = "t{}".format(contador)
      if self.izq.info not in ["+","-","*","/"] and self.der.info not in ["+","-","*","/"]:
        lista.append(variable + ":"+" "+ self.izq.info + " " + self.info + " " + self.der.info)
        contador=contador+1
      elif self.izq.info not in ["+","-","*","/"] and self.der.info in ["+","-","*","/"]:
        lista.append(variable+":"+" "+ self.izq.info + " " +self.info+ " "+ v2)
        contador=contador+1
      elif self.izq.info in ["+","-","*","/"] and self.der.info not in ["+","-","*","/"]:
        lista.append(variable+":"+ " "+ v1+ " "+ self.info+ " "+ self.der.info)
        contador=contador+1
      else:
        lista.append(variable +":"+ " " + v1 +" "+ self.info + " " +v2)
        contador=contador+1
      return variable


class Nodoe:
    def __init__(self, info):
        self.info=info
    def postorden(self):

        return self.info

class Arbol:
    def __init__(self,formula):
        global k
        global s
        s=formula+";" # agregamos una marca de fin de la entrada
        k=0 # indica próximo caracter por procesar
        # definimos funciones para analizar la fórmula
        def expresion(): # retorna puntero a la raíz de un árbol que representa a la fórmula s
            global k
            global s
            a=factor()
            while s[k]=="+" or s[k]=="-":
                op=s[k]
                k+=1
                b=factor()
                a=Nodoi(a,op,b)
            return a
        def factor():
            global k
            global s
            a=termino()
            while s[k]=="*" or s[k]=="/":
                op=s[k]
                k+=1
                b=termino()
                a=Nodoi(a,op,b)
            return a
        def termino(): # posible constante, variable o formula parentizada
            global k
            global s
            if s[k].isalpha() or s[k].isdigit():
                a=Nodoe(s[k])
                k+=1
                return a
            if s[k]=="(": # fórmula parentizada
                k+=1
                a=expresion()
                if s[k]!=")":
                    print("Error: Falta cierra paréntesis: "+formula[k:])
                    assert False
                k+=1
                return a
            print("Error: Falta variable, número o abre paréntesis: "+formula[k:])
            assert False
              
        a=expresion()
        if s[k]!=";":
            print("Error: Basura al final de la fórmula: "+formula[k:])
            assert False                
        self.raiz=a
            
    def postorden(self):
        self.raiz.postorden()
        print()
    
    def codigo(self):
      global lista
      global contador
      contador=1
      lista=[]
      self.postorden()
      if self.raiz.info not in ["+","-","*","/"]:
        lista=["t1"+":"+" "+ str(self.raiz.info)]
        return lista
      else:
        return lista

## Resultados

En esta sección probamos la generación de código que usted tiene que escribir. Mientras usted no sustituya la función ``codigo`` provisoria que viene en la definición de la clase, va a aparecer la lista que generamos ahí. Primero vamos a definir una función ``probar`` que genera el código a partir de una fórmula dada e imprime el resultado:

In [0]:
def probar(formula):
    a=Arbol(formula)
    for x in a.codigo():
        print(x)

A continuación probamos con varias fórmulas. Además de las fórmulas dadaas, usted tiene que agregar tres ejemplos adicionales.

In [3]:
probar("a+1")


t1: a + 1


In [4]:
probar("(a+b)*(c-d)")


t1: a + b
t2: c - d
t3: t1 * t2


In [5]:
probar("(2-p*q)*(1/n+1/(p+q))")


t1: p * q
t2: 2 - t1
t3: 1 / n
t4: p + q
t5: 1 / t4
t6: t3 + t5
t7: t2 * t6


In [6]:
probar("((((a+b)+c)+d)+e)")


t1: a + b
t2: t1 + c
t3: t2 + d
t4: t3 + e


In [7]:
probar("(a+(b+(c+(d+e))))")


t1: d + e
t2: c + t1
t3: b + t2
t4: a + t3


In [8]:
probar("a")


t1: a


In [9]:
probar("a+(b+c)/5")


t1: b + c
t2: t1 / 5
t3: a + t2


In [10]:
probar("p+i+a*(c+o+n+t+r+e+r+a+s)")


t1: p + i
t2: c + o
t3: t2 + n
t4: t3 + t
t5: t4 + r
t6: t5 + e
t7: t6 + r
t8: t7 + a
t9: t8 + s
t10: a * t9
t11: t1 + t10


In [11]:
probar("(i*7)/n+3*(q-(2+c))")


t1: i * 7
t2: t1 / n
t3: 2 + c
t4: q - t3
t5: 3 * t4
t6: t2 + t5


In [12]:
probar("k/n*3+(o+d)")


t1: k / n
t2: t1 * 3
t3: o + d
t4: t2 + t3


In [13]:
probar("a/b*c")


t1: a / b
t2: t1 * c


La estrategia que se usó para que el código funcionara fue principalmente cambiar la función postorden del Nodo interno. 

La función del Nodo externo se modificó de manera que siempre retorna la información del nodo, ya que la información del nodo externo es siempre un número y una variable, por lo que no hay donde más seguir buscando. Los nodos externos son constuidos puramente por variables y por números, nunca poseen los operadores.

La función del Nodo interno se modificó de tal forma que comparaba los nodos en que se encontraba, para lo cual hubo 4 casos. 

* En los dos nodos en que estoy parado son distintos a los operadores, si es así, entonces agrego a la lista variable, operador, variable. Y aumenta contador.

* Si el nodo izquierdo es distinto a los operadores y en el nodo derecho es operador, entonces aplico postorden en el nodo derecho y guardo variable agregando a la lista, esto es, variable, operador, self.der.postorden(). Y aumenta contador.

* Si el nodo izquierdo es operador y el nodo derecho es distinto a operador, entonces, aplico postorden en el nodo izquierdo y guardo variable agregando a la lista, esto es, self.izq.postorden(), operador, variable. Y aumenta contador.

* Finalmente, si ambos nodos son operador, entonces en ambos nodos aplico postorden, guardo variable y agrego a lista de la siguiente manera self.izq.postorden(), operador, self.der.postorden(). Y aumenta contador.

Se definió dos variables globales que se usarían tanto en la función postorden del nodo interno como en la función código, éstas son las variables lista y contador. Lista es una variable tal que se va agregando a ésta lista las operaciones deseadas, y con contador se va aumentando el número de operaciones y permite ponerles nombre.

Se buscó una forma de que la variable "t" se fuera formateando y obteniendo distintos valores para ésta. Se usó la función .format(contador). Contador va en paréntesis de format ya que va cambiando con la cantidad de operaciones que se tiene.

Para una lectura más fácil se definió v1=self.izq.postorden() y v2=self.der.postorden().

La función total retorna variable, donde variable es "t"+ contador. Retorna variable ya que no es una variable global y además es lo que también se va agregando a la lista.

En la función código se tiene dos variables globales también, éstas son lista y contador, además de una lista vacía que se va llenando con la función postorden de Nodo interno. Si la raíz no es un operador, es decir el árbol posee un solo Nodo, entonces retorna una lista que posee "t1"+":"+información del nodo, por el contrario si la raíz del árbol es una operación, es decir para cualquier caso con más de un nodo el árbol, entonces vuelve a Nodo interno, función postorden y llena la lista hasta obtener todos los pasos de la operación.

## Discusión

Suponga que hay escasez de variables temporales, así que se le ha pedido que trate de reciclarlas lo más posible, y solo crear nuevas variables ``ti`` cuando sea estrictamente necesario. Por ejemplo, para la fórmula

```
((a+b)*c)+((d-e)/f)
```

si generamos código como lo hemos hecho en los ejemplos anteriores (código de la izquierda), se usan 5 variables temporales, pero el código de la derecha muestra que se puede evaluar usando solo 2:

```
t1=a+b               t1=a+b
t2=t1*c              t1=t1*c
t3=d-e               t2=d-e
t4=t3/f              t2=t2/f
t5=t2+t4             t1=t1+t2
```

Discuta cómo se podría modificar su algoritmo para que (1) use menos variables temporales, y luego (2) para que use el mínimo posible de variables temporales.

No es necesario que implemente nada, solo que discuta cómo se podría hacer.

**RESPUESTA**

Una forma para disminuir la cantidad de variables temporales que se obtiene es ir viendo en la operación obtenida si está la variable temporal anterior. Si en la nueva variable temporal, dentro de la operación está la variable temporal anterior, entonces en vez de cambiar el nombre de la variable, reemplazo el valor obtenido con la nueva operación y con el mismo valor de la variable temporal anterior, se guarda en una nueva.

Por ejemplo:

````
((a+b)*c)+((d-e)/f)
````


* *t1: a+b*

* t2: t1*c --> Se puede notar que dentro de ésta operación está la variable temporal anterior. Por lo tanto reemplazo su valor en la variable temporal anterior.

  ---> **t1: t1xc**

* *t2: d-e*

* t3: t2/f --> Se puede notar que dentro de ésta operación está la variable temporal anterior. Por lo tanto reemplazo su valor en la variable temporal anterior.

  ---> **t2: t2/f**

* *t3: t1+t2*  --> Se puede notar que dentro de ésta operación está la variable temporal anterior. Por lo tanto reemplazo su valor en la primera variable temporal.

  --->**t1: t1+t2**

Otro ejemplo:

````
(a+(b+(c+(d+e))))
````

Se lee desde dentro de los paréntesis hacia afuera:

* t1: d+e
* t2: t1 +c --> Se puede observar que dentro de ésta variable está la variable temporal anterior --> t1: t1+c
* t1 : t1+b (sucede lo mismo que en el punto anterior)
* t1 : t1+a (sucede lo mismo que en punto ante-anterior)

Para encontrar el mínimo se podría ver los dos paréntesis más grandes que estuviesen al lado de un operador (el operador es entre dos paréntesis lo más grande posible, operador entre medio). Esto es, si existe un paréntesis grande, operador, otro paréntesis grande, entonces la cantidad de variables temporales debería ser 2.

Ejemplo:

````
((a+b)*c)+((d-e)/f)
````

En éste caso se pueden visualizar dos paréntesis "grandes", éstos son 1: ((a+b)*c) y 2: ((d-e)/f) y un operador + al medio, por lo tanto debiesen exitir dos variables temporales, éstas son:

* t1: ((a+b)*c)
* t2: ((d-e)/f)

Juntamos ambas variables temporales en una:

* t1: t1+t2


Otro ejemplo:

````
(a+(b+(c+(d+e))))
````
En éste caso se puede visualizar que no existen paréntesis "grandes", donde al medio poseen un operador, por lo que en éste caso se tendría una variable temporal.

* t1: (d+e)
* t1: c+t1
* t1: b+t1
* t1: a+t1


Otro ejemplo:

````
(2-p*q)*(1/n+1/(p+q))
````

En éste caso, se puede ver que se obtienen dos paréntesis grandes, donde al centro existe un operador, éstas son:

* t1: (2-p*q)
* t2: (1/n+1/(p+q))

Juntamos ambas variables temporales en una:

* t1: t1*t2